In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"inizar","key":"b6e80fd5a31fbec41bb9cfa2ac91a98b"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d whenamancodes/predict-diabities

Dataset URL: https://www.kaggle.com/datasets/whenamancodes/predict-diabities
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 7.73MB/s]


In [4]:
!unzip predict-diabities.zip -d predict-diabities
!ls predict-diabities

Archive:  predict-diabities.zip
  inflating: predict-diabities/diabetes.csv  
diabetes.csv


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [10]:
data = pd.read_csv("/content/predict-diabities/diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
data.drop(columns=["Insulin"], inplace=True)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   BMI                       768 non-null    float64
 5   DiabetesPedigreeFunction  768 non-null    float64
 6   Age                       768 non-null    int64  
 7   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 48.1 KB


In [13]:
X = data.drop("Outcome",axis=1)
Y = pd.get_dummies(data['Outcome'])

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

In [15]:
print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of Y_train is: {Y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of Y_test is: {Y_test.shape}")

Shape of X_train is: (576, 7)
Shape of Y_train is: (576, 2)

Shape of X_test is: (192, 7)
Shape of Y_test is: (192, 2)


In [16]:
input_dim = X.shape[1]

In [17]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [18]:
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 64        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
result = model.fit(X_train, Y_train, epochs = 15, batch_size = 15)

Epoch 1/15
39/39 [==============================] - 0s 4ms/step - loss: 0.5828 - accuracy: 0.7066
Epoch 2/15
39/39 [==============================] - 0s 5ms/step - loss: 0.5738 - accuracy: 0.7066
Epoch 3/15
39/39 [==============================] - 0s 4ms/step - loss: 0.5757 - accuracy: 0.7205
Epoch 4/15
39/39 [==============================] - 0s 4ms/step - loss: 0.5731 - accuracy: 0.7118
Epoch 5/15
39/39 [==============================] - 0s 4ms/step - loss: 0.5695 - accuracy: 0.7170
Epoch 6/15
39/39 [==============================] - 0s 2ms/step - loss: 0.5723 - accuracy: 0.7170
Epoch 7/15
39/39 [==============================] - 0s 2ms/step - loss: 0.5681 - accuracy: 0.7118
Epoch 8/15
39/39 [==============================] - 0s 2ms/step - loss: 0.5691 - accuracy: 0.7153
Epoch 9/15
39/39 [==============================] - 0s 2ms/step - loss: 0.5672 - accuracy: 0.7101
Epoch 10/15
39/39 [==============================] - 0s 2ms/step - loss: 0.5645 - accuracy: 0.7205
Epoch 11/15
39/39 [

In [24]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

6/6 [==============================] - 0s 2ms/step


In [25]:
accuracy = accuracy_score(Y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.71875


In [26]:
model.save('./diabetes.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
model = tf.keras.models.load_model('./diabetes.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./diabetes.tflite", "wb").write(tflite_model)

2548